In [18]:
import sqlite3
import pandas as pd

# Conectando ao banco de dados SQLite
caminho_db = "../database.db"
conexao = sqlite3.connect(caminho_db)

# Cria um dataframe para cada tabela
cpu = pd.read_sql_query("SELECT * FROM cpu", conexao)
memory = pd.read_sql_query("SELECT * FROM memory", conexao)
hd = pd.read_sql_query("SELECT * FROM hd", conexao)
ssd = pd.read_sql_query("SELECT * FROM ssd", conexao)
psu = pd.read_sql_query("SELECT * FROM psu", conexao)
computerCase = pd.read_sql_query("SELECT * FROM computerCase", conexao)
cooler = pd.read_sql_query("SELECT * FROM cooler", conexao)
gpu = pd.read_sql_query("SELECT * FROM gpu", conexao)
motherboard = pd.read_sql_query("SELECT * FROM motherboard", conexao)

# Fechando a conexão
conexao.close()


In [19]:
cpu = cpu.drop_duplicates()
memory = memory.drop_duplicates()
hd = hd.drop_duplicates()
ssd = ssd.drop_duplicates()
psu = psu.drop_duplicates()
computerCase = computerCase.drop_duplicates()
cooler = cooler.drop_duplicates()
gpu = gpu.drop_duplicates()
motherboard = motherboard.drop_duplicates()

In [20]:
motherboard = motherboard[~(
    (motherboard['extensionPCI3x16'] == 0) &
    (motherboard['extensionPCI4x16'] == 0)
)]

motherboard['socket'] = (
    motherboard['socket']
    .str.split(',')
    .apply(lambda lst: [s.strip() for s in lst])
)
motherboard = motherboard.explode('socket').reset_index(drop=True)

invalid = ['SoC', 'AM1', '775', '2011-v3', '2011-3']
motherboard = motherboard[~motherboard['socket'].isin(invalid)]

In [21]:
cpu['socket'] = (
    cpu['socket']
    .str.split(',')
    .apply(lambda lst: [s.strip() for s in lst])
)

cpu = cpu.explode('socket').reset_index(drop=True)

invalid_sockets = ['FM2', 'AM3', 'sWRX80', 'LGA 1851', '1851']
cpu = cpu[~cpu['socket'].isin(invalid_sockets)]

In [22]:
# Se a coluna 'memoryType' do dataframe motherboard está com valores ['DDR4, SO - DIMM', 'DDR4, ECC - Speicher'], a mesma deve ser alterada para 'DDR4'
substituir = ['DDR4, SO - DIMM', 'DDR4, ECC - Speicher']
motherboard = motherboard.replace({'memoryType': {k: 'DDR4' for k in substituir}})

In [23]:
# Vamos dar split na coluna 'type' de memory com '-' e manteremos apenas o primeiro item
memory['type'] = memory['type'].str.split('-').str[0]
memory = memory[~memory['type'].isin(['DDR1', 'DDR2', 'DDR3L'])]

In [24]:
# Modificar o valor da coluna 'adicionalFan' de 0 -> false e 1 -> true
cooler['adicionalFan'] = cooler['adicionalFan'].map({0: False, 1: True})

In [25]:
cooler['sockets'] = cooler['sockets'].str.split(',')

cooler_exploded = cooler.explode('sockets')

df_assoc = pd.merge(
    cpu[['mpn', 'socket']], 
    cooler_exploded[['mpn', 'sockets']], 
    left_on='socket', 
    right_on='sockets',
    how='inner'
)

CPU_Cooler = df_assoc.rename(columns={
    'mpn_x': 'cpu_mpn',
    'mpn_y': 'cooler_mpn'
})[['cpu_mpn', 'cooler_mpn']].drop_duplicates().reset_index(drop=True)

In [26]:
df_assoc = pd.merge(
    cpu[['mpn', 'socket']],
    motherboard[['mpn', 'socket']],
    on='socket',
    how='inner'
)

CPU_PlacaMae = (
    df_assoc
    .rename(columns={'mpn_x': 'cpu_mpn', 'mpn_y': 'placamae_mpn'})
    [['cpu_mpn', 'placamae_mpn']]
    .drop_duplicates()
    .reset_index(drop=True)
)


In [27]:
df_assoc = pd.merge(
    motherboard[['mpn', 'memoryType']],
    memory[['mpn', 'type']],
    left_on='memoryType',
    right_on='type',
    how='inner'
)

PlacaMae_MemoriaRAM = (
    df_assoc
    .rename(columns={
        'mpn_x': 'placamae_mpn',
        'mpn_y': 'memoriaram_mpn'
    })
    [['placamae_mpn', 'memoriaram_mpn']]
    .drop_duplicates()
    .reset_index(drop=True)
)

In [30]:
usuario = 'postgres'
senha = '843324'
host = 'localhost'
porta = '5432'
novo_banco = banco = 'pc_assembly'
caminho_sql = '../modelo_pc_assembly.sql'

In [31]:
import psycopg2

con = psycopg2.connect(
    dbname='postgres',
    user=usuario,
    password=senha,
    host=host,
    port=porta
)
con.autocommit = True
cur = con.cursor()
try:
    cur.execute(f'CREATE DATABASE {novo_banco};')
except psycopg2.Error:
    print("Banco de dados já existe")
cur.close()
con.close()

con = psycopg2.connect(
    dbname=novo_banco,
    user=usuario,
    password=senha,
    host=host,
    port=porta
)
cur = con.cursor()
with open(caminho_sql, 'r', encoding='utf-8') as f:
    sql_script = f.read()
cur.execute(sql_script)  # Executa todo o script de uma vez
con.commit()
cur.close()
con.close()

print('Banco de dados e tabelas criados com sucesso!')

Banco de dados já existe
Banco de dados e tabelas criados com sucesso!


In [32]:
from sqlalchemy import create_engine

engine = create_engine(
    f"postgresql+psycopg2://{usuario}:{senha}@{host}:{porta}/{banco}",
    echo=False
)

pk_cols = {
    'CPU': 'mpn',
    'GPU': 'mpn',
    'HD': 'mpn',
    'MemoriaRAM': 'mpn',
    'PlacaMae': 'mpn',
    'FonteDeAlimentacao': 'mpn',
    'SSD': 'mpn',
    'CPUCooler': 'mpn',
    'CPU_Cooler': ['cpu_mpn','cooler_mpn'],
    'CPU_PlacaMae': ['cpu_mpn','placamae_mpn'],
    'PlacaMae_MemoriaRAM': ['placamae_mpn','memoriaram_mpn'],
}

dfs_para_inserir = {
    'CPU': cpu,
    'GPU': gpu,
    'HD': hd,
    'MemoriaRAM': memory,
    'PlacaMae': motherboard,
    'FonteDeAlimentacao': psu,
    'SSD': ssd,
    'CPUCooler': cooler,
    'CPU_Cooler': CPU_Cooler,
    'CPU_PlacaMae': CPU_PlacaMae,
    'PlacaMae_MemoriaRAM': PlacaMae_MemoriaRAM
}

for tabela, df in dfs_para_inserir.items():
    pk = pk_cols[tabela]
    
    if isinstance(pk, list):
        select_expr = "||'|'||".join(pk)
        sql = f'SELECT {select_expr} AS pk FROM desktop."{tabela}"'
        existentes = pd.read_sql(sql, engine)['pk'].tolist()
        df['_pk'] = df[pk].agg('|'.join, axis=1)
        df_novo = df[~df['_pk'].isin(existentes)].drop(columns=['_pk'])
    else:
        sql = f'SELECT "{pk}" FROM desktop."{tabela}"'
        existentes = pd.read_sql(sql, engine)[pk].tolist()
        df_novo = df[~df[pk].isin(existentes)]
    
    if not df_novo.empty:
        df_novo.to_sql(
            name=tabela,
            con=engine,
            schema='desktop',
            if_exists='append',
            index=False,
            method='multi',
            chunksize=500
        )
        print(f"✔ Inseriu {len(df_novo)} novos em {tabela}")
    else:
        print(f"– Não há registros novos para {tabela}")


– Não há registros novos para CPU
– Não há registros novos para GPU
– Não há registros novos para HD
– Não há registros novos para MemoriaRAM
– Não há registros novos para PlacaMae
– Não há registros novos para FonteDeAlimentacao
– Não há registros novos para SSD
– Não há registros novos para CPUCooler
– Não há registros novos para CPU_Cooler
– Não há registros novos para CPU_PlacaMae
– Não há registros novos para PlacaMae_MemoriaRAM
